In [ ]:
# # the imports in this cell are required when running on local device
# import os, sys
# sys.path.append(os.path.join('..', '..'))
# from utils.applyML_util import train_regression, eval_regression
# from utils.featureSelection_util import (pearson_correlation_fs, 
#                                          seleckKBest_fs, selectSequential_fs)

In [10]:
# the imports in this cell are required when running from Cloud (Colab/Kaggle)
# before running on cloud you nee to upload the .py files
# from 'Notebooks/utils' directory
from applyML_util import train_regression, eval_regression, showEvalutationGraph_regression
from featureSelection_util import (pearson_correlation_fs, 
                                   seleckKBest_fs, selectSequential_fs)

# New Section

**XGBoost Documentation link:** https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor <br>
**SK-Lego Documentation link:** https://scikit-lego.netlify.app/meta.html#Zero-Inflated-Regressor

In [ ]:
!pip install sklego

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklego.meta import ZeroInflatedRegressor #!pip install sklego
from sklearn.svm import SVC
from xgboost.sklearn import XGBRegressor

In [6]:
# global random seed
RAND_SEED = 42

ziReg = ZeroInflatedRegressor(classifier=SVC(), regressor=XGBRegressor(random_state=RAND_SEED, n_jobs=-1, objective ='reg:squarederror'))

# initial model with only random seed and not any hyper-parametes
initial_model = ziReg

# hyper-parameters
classifier__kernel = ['poly']
classifier__C = [8]
classifier__degree = [5]
classifier__gamma = [1]

regressor__n_estimators = [1, 5, 10]
#regressor__max_depth = [max(1, x*3) for x in range(0, 11)]
regressor__early_stopping_rounds = [5]
regressor__learning_rate = [0.1, 1.0]
regressor__gamma = [0, 5, 10]
regressor__subsample = [ 0.5, 1.0]


param_grid = {'classifier__kernel':classifier__kernel,'classifier__gamma':classifier__gamma,'classifier__degree': classifier__degree,'classifier__C':classifier__C,
              'regressor__n_estimators':regressor__n_estimators, 'regressor__learning_rate':regressor__learning_rate,
              'regressor__gamma': regressor__gamma, 'regressor__subsample': regressor__subsample, 'regressor__early_stopping_rounds': regressor__early_stopping_rounds}

# variables needed for showEvalGraph_regression() function
# MODEL_CLASS = ziReg
# x_axis_param_name = 'regressor__C' #???classifier__C
# x_axis_vals = regressor__C

In [ ]:
ziReg.get_params().keys()


dict_keys(['classifier__C', 'classifier__break_ties', 'classifier__cache_size', 'classifier__class_weight', 'classifier__coef0', 'classifier__decision_function_shape', 'classifier__degree', 'classifier__gamma', 'classifier__kernel', 'classifier__max_iter', 'classifier__probability', 'classifier__random_state', 'classifier__shrinking', 'classifier__tol', 'classifier__verbose', 'classifier', 'regressor__base_score', 'regressor__booster', 'regressor__colsample_bylevel', 'regressor__colsample_bynode', 'regressor__colsample_bytree', 'regressor__gamma', 'regressor__importance_type', 'regressor__learning_rate', 'regressor__max_delta_step', 'regressor__max_depth', 'regressor__min_child_weight', 'regressor__missing', 'regressor__n_estimators', 'regressor__n_jobs', 'regressor__nthread', 'regressor__objective', 'regressor__random_state', 'regressor__reg_alpha', 'regressor__reg_lambda', 'regressor__scale_pos_weight', 'regressor__seed', 'regressor__silent', 'regressor__subsample', 'regressor__verbo

## 1. Experimentation on the Weather Daily dataset

In [7]:
# Load the train dataset
weather_daily_train_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/train/brri-weather_train_regression.csv')

# Load the test set
weather_daily_test_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/test/brri-weather_test_regression.csv')

In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 0, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 10, 'regressor__subsample': 1.0}
Train set performance: r2-score=0.1807, mae=5.3438, rmse=13.5484


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1159, mae=5.6485, rmse=16.2581


### 1.1 Apply Pearson Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs1_train_df, cols_to_drop = pearson_correlation_fs(weather_daily_train_df, 'Rainfall (mm)')

# keep only selected features on the test dataset
weather_daily_fs1_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

dropping Sunshine (hour/day) from (Cloudy (hour/day), Sunshine (hour/day))


In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs1_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 0, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 10, 'regressor__subsample': 0.5}
Train set performance: r2-score=0.1963, mae=5.2991, rmse=13.4239


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs1_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs1_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1229, mae=5.5577, rmse=16.194


### 1.2 Apply SelectKBest Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs2_train_df, cols_to_drop = seleckKBest_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs2_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Max Temp. (degree Celcius)', 'Relative Humidity (morning, %)']


In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs2_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 0, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 10, 'regressor__subsample': 1.0}
Train set performance: r2-score=0.1848, mae=5.302, rmse=13.5114


In [ ]:
# r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs2_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs2_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1226, mae=5.4867, rmse=16.1964


### 1.3 Apply SelectSequential Feature Selection to Daily Weather Dataset

In [11]:
# select features from the train dataset
weather_daily_fs3_train_df, cols_to_drop = selectSequential_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs3_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Actual Evaporation (mm)', 'Relative Humidity (morning, %)']


In [12]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs3_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__C': 8, 'classifier__degree': 5, 'classifier__gamma': 1, 'classifier__kernel': 'poly', 'regressor__early_stopping_rounds': 5, 'regressor__gamma': 0, 'regressor__learning_rate': 0.1, 'regressor__n_estimators': 10, 'regressor__subsample': 0.5}
Train set performance: r2-score=0.1831, mae=5.3077, rmse=13.5454


In [ ]:
# r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs3_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [13]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs3_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1101, mae=5.6359, rmse=16.311
